# Scraper für WGgesucht - München

Ausgangslink: https://www.wg-gesucht.de/wg-zimmer-in-Muenchen.90.0.1.0.html

Bibliotheken installieren


In [117]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

First part of the code gets all Announces in the specified pages (10) by URL and saves it to a list

In [118]:
def api_call(url):
  payload = {'api_key': 'cbff1dee043e85253ef98571a5a7a17a', 'url': url}
  try:
    response = requests.get('http://api.scraperapi.com', params=payload)
  except ConnectionError:
    print("Connection Error")
    exit(0)
  
  return response

In [119]:
page_amount = 25

url_first_part = 'https://www.wg-gesucht.de/wg-zimmer-in-Muenchen.90.0.1.'
url_second_part = '.html'                                                                       #complete url => first part + page_number + second part

total_link_list = []

for page_number in range(1, page_amount):

    response = api_call(url_first_part + str(page_number) + url_second_part)
    soup = BeautifulSoup(response.content, "html.parser")

    relevant_page_content = soup.find("div", attrs={'class': 'col-xs-12 col-md-8 col-md-push-4'})
    links = relevant_page_content.find_all("a", attrs={'class': 'detailansicht'})

    even = True
    for link in links[1:]:
        intern_link = link.get("href")
        if even:
            if "sort_column" not in intern_link:
                total_link_list.append("https://www.wg-gesucht.de/" + intern_link)
            even = False
        else:
            even = True

print(total_link_list)


['https://www.wg-gesucht.de/wg-zimmer-in-Muenchen-Laim.10983807.html', 'https://www.wg-gesucht.de/wg-zimmer-in-Muenchen-Moosach.3207396.html', 'https://www.wg-gesucht.de/wg-zimmer-in-Muenchen-Sendling-Westpark.11009000.html', 'https://www.wg-gesucht.de/wg-zimmer-in-Muenchen.11005071.html', 'https://www.wg-gesucht.de/wg-zimmer-in-Muenchen-Maxvorstadt.11005484.html', 'https://www.wg-gesucht.de/wg-zimmer-in-Muenchen-Neuhausen-Nymphenburg.10979833.html', 'https://www.wg-gesucht.de/wg-zimmer-in-Muenchen-Feldmoching-Hasenbergl.11001914.html', 'https://www.wg-gesucht.de/wg-zimmer-in-Muenchen-Ramersdorf-Perlach.10992217.html', 'https://www.wg-gesucht.de/wg-zimmer-in-Muenchen-80686.10982509.html', 'https://www.wg-gesucht.de/wg-zimmer-in-Muenchen-Maxvorstadt.11005483.html', 'https://www.wg-gesucht.de/wg-zimmer-in-Muenchen-Maxvorstadt.9272407.html', 'https://www.wg-gesucht.de/wg-zimmer-in-Muenchen-Laim.11000568.html', 'https://www.wg-gesucht.de/wg-zimmer-in-Muenchen-Germering.9321577.html', 'http

In [120]:
def get_page_content(page_content, Url):
    try:
        title = page_content.find("h1", attrs={'class': 'headline headline-detailed-view-title'}).text.strip()             # get title String
        
        size = page_content.find_all("b", attrs={'class': 'key_fact_value'})[0].text.strip()             # get size String

        price_total = page_content.find_all("b", attrs={'class': 'key_fact_value'})[1].text.strip()             # get price_total String

        price = page_content.find_all("span", attrs={'class': 'section_panel_value'})[0].text.strip()             # get price String

        extra_costs = page_content.find_all("span", attrs={'class': 'section_panel_value'})[1].text.strip()             # get extra_costs String

        other_costs = page_content.find_all("span", attrs={'class': 'section_panel_value'})[2].text.strip()             # get other_costs String

        deposit = page_content.find_all("span", attrs={'class': 'section_panel_value'})[3].text.strip()             # get deposit String

        redemption_agreement = page_content.find_all("span", attrs={'class': 'section_panel_value'})[4].text.strip()             # get redemption_agreement String

        date_free = page_content.find_all("span", attrs={'class': 'section_panel_value'})[5].text.strip()             # get date_free String

        date_free_till= "n.a."
        if len(page_content.find_all("span", attrs={'class': 'section_panel_value'})) > 6:
            date_free_till = page_content.find_all("span", attrs={'class': 'section_panel_value'})[6].text.strip()             # get date_free_till String
               
        online_since =  page_content.find("b", attrs={'class': 'noprint'}).text.strip()             # get online_since String
        
        adress = page_content.find_all("span", attrs={'class': 'section_panel_detail'})[5].text.strip()             # get adress String

        details = ""
        for detail in page_content.find_all("ul", attrs={'class': 'pl15'})[0].find_all("span", attrs={'class': 'section_panel_detail'}):              #get Details String
            details += detail.text.strip() + '\n'

        looking_for = ""
        for looking_for_element in page_content.find_all("ul", attrs={'class': 'pl15'})[1].find_all("span", attrs={'class': 'section_panel_detail'}):              #get looking_for String
            looking_for += looking_for_element.text.strip() + '\n'

        amount_of_icons = "n.a. "
        if len(page_content.find_all("div", attrs={'class': 'utility_icons'})) > 0:
            amount_of_icons = len(page_content.find_all("div", attrs={'class': 'utility_icons'})[0].find_all("div", attrs={'class': 'text-center'}))         # get amount_of_icons String
        
        amount_of_icons_needed = "n.a."
        if len(page_content.find_all("div", attrs={'class': 'utility_icons'})) > 1:
            counter = 0
            counter += len(page_content.find_all("div", attrs={'class': 'utility_icons'})[1].find_all("div", attrs={'class': 'text-center'}))         # get amount_of_icons String
            counter += len(page_content.find_all("div", attrs={'class': 'utility_icons'})[1].find_all("a", attrs={'class': 'text-center campaign_click'}))
            amount_of_icons_needed = counter

        description_room_div = page_content.find("div", attrs={'id': 'freitext_0'})
        description_room = 'n.a.'
        if description_room_div:
            description_room = description_room_div.text.strip()
                    
        description_location_div = page_content.find("div", attrs={'id': 'freitext_1'})
        description_location = 'n.a.'
        if description_location_div:
            description_location = description_location_div.text.strip()
      
        # Check and get description_life if it exists
        description_life_div = page_content.find("div", attrs={'id': 'freitext_2'})
        description_life = 'n.a.'
        if description_life_div:
            description_life = description_life_div.text.strip()
       
        # Check and get description_other if it exists
        description_other_div = page_content.find("div", attrs={'id': 'freitext_3'})
        description_other = 'n.a.'
        if description_other_div:
            description_other = description_other_div.text.strip()
                    

        return {'Url': Url, 'Title': title, 'Size': size, 'PriceTotal': price_total, 'Price': price, 'ExtraCosts': extra_costs, 'OtherCosts': other_costs, 'Deposit': deposit, 'RedemptionAgreement': redemption_agreement, 'DateFree': date_free, 'DateFreeTill': date_free_till, 'OnlineSince': online_since, 'Adress': adress, 'Details': details, 'LookingFor': looking_for, 'AmountofIcons': amount_of_icons, 'AmountofIconsNeeded': amount_of_icons_needed, 'DescriptionRoom': description_room, 'DescriptionLocation': description_location, 'DescriptionLife': description_life, 'description_other': description_other}
    except (AttributeError, IndexError) as e:
        return "error"


In [121]:
main_dict = {'Url': [], 'Title': [], 'Size': [], 'PriceTotal': [], 'Price': [], 'ExtraCosts': [], 'OtherCosts': [], 'Deposit': [], 'RedemptionAgreement': [], 'DateFree': [], 'DateFreeTill': [], 'OnlineSince': [], 'Adress': [], 'Details': [], 'LookingFor': [], 'AmountofIcons': [], 'AmountofIconsNeeded': [], 'DescriptionRoom': [], 'DescriptionLocation': [], 'DescriptionLife': [], 'description_other': []}

for link in total_link_list:
    
    response = api_call(link)
    soup = BeautifulSoup(response.content, "html.parser")
    temp_dict = get_page_content(soup, link)
    if temp_dict == "error":
        continue    
    for key, value in temp_dict.items():
        if key in main_dict:
            main_dict[key].append(value) 
        else:
            main_dict[key] = [value]  

    
result_dataframe = pd.DataFrame(main_dict)

print(result_dataframe.size)
display(result_dataframe)

# Save DataFrame to CSV without the index
result_dataframe.to_csv('wggesucht_scraped_16_05.csv', index=False)   


10500


,Url,Title,Size,PriceTotal,Price,ExtraCosts,OtherCosts,Deposit,RedemptionAgreement,DateFree,...,OnlineSince,Adress,Details,LookingFor,AmountofIcons,AmountofIconsNeeded,DescriptionRoom,DescriptionLocation,DescriptionLife,description_other
0,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,10m² Zimmer in entspannter 4er Wg :),10m²,690€,690€,n.a.,n.a.,1500€,500€,01.06.2024,...,40 Minuten,Joergstraße 84\n ...,10m²\n \n ...,Geschlecht egal\n ...,6,4,Hallo du:) \n Dein neues Zimmer ist angenehme ...,Unsere WG ist circa 8 Gehminuten vom Laimer Pl...,"Wir sind Francesco, Dan und Kibo :) Unsere Lie...","Erzähl mir gern etwas über dich, sodass wir sc..."
1,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,10qm Zimmer in Doppelhaushälfte,10m²,690€,590€,100€,0€,900€,0€,01.08.2024,...,45 Minuten,gleißmüllerstr. 9\n ...,10m²\n \n ...,Mann\n ...,11,2,Unsere WG hat 6 Zimmer im Grünen und liegt an ...,n.a.,n.a.,n.a.
2,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,Helles Zimmer in 4-er WG,17m²,755€,755€,n.a.,n.a.,n.a.,n.a.,01.07.2024,...,45 Minuten,Holsteiner Straße\n ...,17m²\n \n ...,Geschlecht egal\n,5,3,"Hey, \r\n leider muss ich Ende Juni aus meiner...",Die Wohnung befindet sich direkt am Südpark un...,"Die WG ist nicht als Zweck-Wg angedacht, sonde...",Im Zimmer steht ein schöner und geräumiger Kle...
3,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,2er Studenten WG. Wohnen neben der Uni. Netter...,10m²,850€,750€,100€,n.a.,n.a.,0€,15.06.2024,...,46 Minuten,Schellingstraße 21\n ...,10m²\n \n ...,Geschlecht egal\n ...,2,n.a.,Bitte beachte: \n Das Zimmer ist rein von der ...,In einem Umkreis von ca. 200m Luftlinie hast d...,"Was ich auf keinen Fall will ist ne Zweck-WG, ...",Schreib mir einfach über wg-gesucht ein paar Z...
4,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,Zimmer zu vermieten in 4-Zimmer-Wohnung in der...,103m²,1160€,1160€,n.a.,n.a.,1000€,n.a.,16.05.2024,...,46 Minuten,Heßstraße\n ...,103m²\n \n ...,Geschlecht egal\n,4,n.a.,Zimmer zu vermieten in 4-Zimmer-Wohnung in der...,munich,n.a.,**Dieser Vermieter akzeptiert keine Studenten!...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,Großes Zimmer im Zentrum in einer 3er WG direk...,20m²,1100€,1100€,0€,0€,3000€,0€,01.07.2024,...,2 Tage,Barerstraße 42\n ...,20m²\n \n ...,Geschlecht egal\n,4,4,Das Zimmer in der Wohnung ist möbiliert mit Be...,n.a.,n.a.,n.a.
496,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,"Groß, zentral, ruhig - ZIMMER in WG am Goethep...",20m²,1150€,1050€,100€,n.a.,2000€,n.a.,01.06.2024,...,2 Tage,Waltherstr.\n ...,20m²\n \n ...,Frau\n ...,10,4,Neu renoviertes und möbliertes WG - Zimmer in ...,"Ruhige Seitenstraße am Goetheplatz, 100m zur U...",Ausstattung \r\n Hochwertig möbliertes Zimmer ...,Mindestmietzeit 6 Monate \n\r\n Modernisierter...
497,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,Mitbewohnerin gesucht - WG-Zimmer 16m² + eigen...,16m²,900€,900€,0€,0€,n.a.,0€,27.05.2024,...,2 Tage,Schäufeleinstr. 32\n ...,16m²\n \n ...,Frau\n,9,4,Die Wohnung befindet sich im 3. und 4. Stock e...,Die Wohnung befindet sich in Laim und ist ca. ...,"Dein zukünftiger Mitbewohner ist Miguel, er is...",n.a.
498,https://www.wg-gesucht.de/wg-zimmer-in-Muenche...,(Verfügbar 1-24 Monate) - Modernes und gemütli...,9m²,1140€,1140€,n.a.,n.a.,n.a.,n.a.,05.08.2024,...,8 Stunden,Wittelsbacherstraße\n ...,9m²\n \n ...,Geschlecht egal\n,4,n.a.,- Handtücher und Bettlaken \n - Hygieneutensil...,Homelike ist eine Online-Buchungsplattform für...,n.a.,Homelike Code: MUC-768972
